# Vigenere chiper cryptoanalysis


This notebook studies the cryptoanalysis of the [Vigenere][wiki] chiper, which is polyalphabetic, namely two occurrences  $a_{1}, a_{2}$ of character $a$ belonging to the plaintext are associated with occurrences $c_{1}, c_{2}$ in the ciphertext, such that $c_{1}\neq c_{2}$ with very high probability.

Our implementation closely follows a class lecture given by Prof. [Orazio Puglisi][puglisi] within Cryptography course at the University of Florence. In order to fully understand the last part, where the concept of *mutual coincidence index* is crucial, we rest on the explanation at pag. 20 of [this notes][notes].

[wiki]:https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher
[puglisi]:http://web.math.unifi.it/users/puglisi/
[notes]:http://iml.univ-mrs.fr/~ritzenth/cours/crypto.general.pdf

In [1]:
import itertools
from itertools import *
from copy import copy, deepcopy
from heapq import *
from random import *

from sympy import *

init_printing()

Let $\mathcal{A}$ be our alphabet composed of standard English characters plus the `space` character. It will be sufficient to encode simple and (not so quite) short messages. To define it as a group $\frac{\mathbb{Z}}{n\mathbb{Z}}$ in the Python language we use a `dict` object, which can be reversed because it is a bijection.

In [2]:
def make_alphabet_entry(i):
    alpha = i + ord('a')
    return chr(alpha),i

A = dict(map(make_alphabet_entry, range(26)))
A.update({' ':26})
inverse_A = {v:k for k,v in A.items()}

A

{' ': 26,
 'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

We define a function `encode` that consumes a string and produces a list of integer in our field. In parallel, function `decode` goes backwards, it consumes a list of integers and return a string. Such functions are useful in order to use the cipher and analyze it using `str` objects.

In [3]:
def encode(s, alphabet=A): return list(map(lambda i: alphabet[i], s))

def decode(e, inverse_alphabet=inverse_A): return "".join(map(lambda i: inverse_alphabet[i], e))

The following plaintext is a prose taken from [here]; before using it we have to swipe out punctuation marks:

[here]:http://www.bartleby.com/209/2.html

In [4]:
with open('plain_text.txt', 'r') as f:
    plain_text = f.read().lower()

remove_chars = [',', '.', ';', ':', '-', '(', ')', "'", '"']
for rc in remove_chars: 
    plain_text = plain_text.replace(rc, '')
plain_text = plain_text.replace('\n', ' ')

plain_text = "".join(filter(lambda c: not c.isdigit(), plain_text))
encoded_plain_text = encode(plain_text)

plain_text, len(plain_text)

('the attraction of medieval literature comes perhaps more strongly from some other countries than from england in france and provence in germany and iceland there were literary adventurers more daring and achievements more distinguished it was not in england that the most wonderful things were produced there is nothing in old english that takes hold of the mind with that masterful and subduing power which still belongs to the lyrical stanzas of the troubadours and minnesingers to welsh romance or to the epic prose of the iceland histories   the norman conquest degraded the english language from its literary rank and brought in a new language for the politer literature it did not destroy in one sense it did not absolutely interrupt english literature but it took away the english literary standard and threw the country back into the condition of italy before dantean anarchy of dialects when a new literary language was established in the time of chaucer the middle ages were nearly over a

With the following assert we ensure that function `decode` is the inverse of function `encode`:

In [5]:
assert decode(encode(plain_text)) == plain_text

Let $k \in \left(\frac{\mathbb{Z}}{n\mathbb{Z}}\right)^{m}$ be a *key* of length $m\in\mathbb{N}$. In order to have a meaningful analysis we build a function `generate_random_key` which generate a random key of random length, keeping it safe...we will uncover it only at the end to check our work:

In [54]:
def generate_random_key(alphabet=A):
    length = randint(0,floor(len(alphabet)/2))
    key = [0] * length
    for i in range(length): key[i] = randint(0, len(alphabet)-1)
    return key

#key = encode("ericsmullyan")
secret_key = generate_random_key(A)

Now we are in the position to define `encrypt` and `decrypt` functions, both of them consumes an *encoded* message, namely a list of integers, and a key:

In [55]:
def encrypt(message, key, alphabet=A):
    n = len(alphabet)
    return [(p+v)%n for p,v in zip(message, cycle(key))]

def decrypt(cipher, key, alphabet=A):
    n = len(alphabet)
    return [(c-v)%n for c,v in zip(cipher, cycle(key))]

the following is the *ciphertext* produced by the application of function `encrypt` to the plaintext:

In [56]:
cipher_text = encrypt(encoded_plain_text, secret_key)
decode(cipher_text)

'rxiordymmtadmcdcwkr pzmqzad zdjmmjbmcpgccpxvavzczewoczw liammck okkm chnmbioedm cqkjscxfzpxveyiiyvvcckjisbiibpmbqqwwztmvzchofbtqqdk yyroxpwhmdfvzchoznjgmdlvrxifvka cvhggiifrbcvmuc liyfvbxvyez ytefzylvmdlvzslxvfjhqdanybsfvkiddjqiejmgypivujhrzhdbededzqmieaebukycmjhofudaecyvhevzcgjibkycudonylifvkum ubyctdhypw lz vldxwzylvudhjjtdtdrqddyhofqxojlp dqpjjtdcwkycqqudltdkzdmveyioybegjpwafbhwltdgkmipudovnd thkacutpvqim bkg xevbqpxcqdm lbfmgse qcywzpinydjojsjvehwp qhckbxvmdlvkyrbvcnisvznyisompqntqzjkqrrvktmljwvrxiov nylfzjqudcwkycqqqycaebukmddjwmguwoqkycqqvjpbebqntibkmnrphtxbfzquhofudtdrqddyhgzckirrjvrhwhyyxgqwnoqhimwpvpdvewzuhxpdyvydedzqivlu oblsbfro yvsfqdm lfwggiifqwnoqhiosgiozdezuuhimidsvcym ohdlpsbvkx zimvgidszoei jhw hs kdjgjqqiruvfk yvqdogghlobty crappudqkdedeqajm dpmlcveymvcck zcmvxza pqvmqcywzuimbpebukyccvdvrxiotzziehfv qgzqtso qaccpgcdonouevvmvdxjlqtlsmamgioulsoqrvvzceftscv whzgqpttdxvhymiyqdbvgegujmmzgboblsbfro ylegqpxomstdqxisqtsveymvryqtqzkvoyipauvojsjvyzlzjudpxpxvhvz yciphwcv lmmyqr

Assuming to know the `secret_key`, we ensure that function `decrypt` is the inverse of function `encrypt`:

In [57]:
assert decode(decrypt(cipher_text, secret_key)) == plain_text

We now define two linguistic objects:

   - let $I_{c}$ be the *coincidence index* of a sequence $\alpha$, over an alphabet $A$, defined as 
   $$ I_{c}(\alpha) = \sum_{i=0}^{n}{\frac{f_{i}(f_{i}-1)}{m(m-1)}} $$ where $n$ is the length of the alphabet, $m$ is the length of $\alpha$ and $f_{i}$ is the frequency of symbol $a_{i}\in A$, namely the number of occurrences of $a_{i}$ in $\alpha$. In other words, $I_{c}$ is the probability to sample two occurrences of the same symbol $a$, forall $a\in A$, from the sequence $\alpha$. 

In [37]:
def frequencies(lst, alphabet=A, inverse_alphabet=inverse_A):
    """ Returns a `dict` counting occcurrences of each object of the alphabet within the iterable.
    
    `frequencies` consumes an iterable $lst$ and an alphabet $A$, 
    produces a dictionary of entries $(k,v)$ where $k$ is a character 
    in $A$ and $v$ is the number of occurrences of $k$ in $lst$"""
    
    freqs = {k:0 for k in alphabet.keys()}
    for c in lst: freqs[inverse_alphabet[c]] += 1
    return freqs

def length_from_frequencies(freqs):
    return sum(v for v in freqs.values())

def coincidence_index(freqs, alphabet=A):
    denom = length_from_frequencies(freqs)
    if denom in range(2): return None
    return sum(v*(v-1) for v in freqs.values())/(denom*(denom-1))
    
def mutual_coincidence_index(fst_freqs, snd_freqs, offset=0, 
                             alphabet=A, inverse_alphabet=inverse_A):
    fst_len = length_from_frequencies(fst_freqs)
    snd_len = length_from_frequencies(snd_freqs)
    n = len(alphabet)
    return sum(fst_freqs[k] * snd_freqs[inverse_alphabet[(v+offset) % n]] 
               for k,v in alphabet.items())/(fst_len * snd_len)

In [58]:
plaintext_frequencies = frequencies(encoded_plain_text)
plaintext_frequencies

{' ': 335,
 'a': 115,
 'b': 19,
 'c': 40,
 'd': 67,
 'e': 215,
 'f': 34,
 'g': 39,
 'h': 81,
 'i': 113,
 'j': 0,
 'k': 9,
 'l': 57,
 'm': 29,
 'n': 140,
 'o': 120,
 'p': 21,
 'q': 1,
 'r': 125,
 's': 93,
 't': 162,
 'u': 35,
 'v': 14,
 'w': 29,
 'x': 0,
 'y': 26,
 'z': 1}

In [59]:
coincidence_index(plaintext_frequencies)

In [60]:
coincidence_index(frequencies(cipher_text))

In [89]:
def spread(message, block_length):
    return [message[i:i+block_length] for i in range(0, len(message), block_length)]
    
def col(spreaded, c, join=False, joiner=lambda c: ''.join(decode(c))):
    column = [lst[c] if c < len(lst) else None for lst in spreaded]
    ready = list(filter(lambda i: i is not None, column))
    return joiner(ready) if join else ready

In [90]:
spreaded = spread(cipher_text, 12)

In [93]:
c = col(spreaded,2,True)
c

'iddgwkixvmhrhiiyelsmdedjidxdxdded mxejhrsridewehdkxvy siidsdsvlddkveiggdiepdbeiqvdirtsiiyimdjilqxddikeerlvmvdveigqdxvdxixvghvvriivd eiiesdirmxifdddhddtddsdgpddi'

In [31]:
def analyze(cipher_text):
    res = {}
    # we discard the case where the key length equals the
    # length of the cipher text, since it is the case of 
    # OneTimePad chiper, which is unbreakable!
    for d in range(2, len(cipher_text)):
        spreaded = spread(cipher_text, d)
        res[d] = []
        for c in range(d):
            column = col(spreaded, c)
            freqs = frequencies(column)
            ci = coincidence_index(freqs)
            if ci: res[d].append(ci)
    return res

def guess_key_length(analysis, threshold=0.06):
    candidates = {}
    for k,v in analysis.items():
        cs = list(filter(lambda i: i > threshold, v))
        if cs and len(cs) > ceiling(k/2): candidates[k] = cs
    return candidates

In [94]:
analysis = analyze(cipher_text)

In [95]:
probing_key_length = 12
guess_key_length(analysis)[probing_key_length]

In [66]:
def build_offsets_eqs(cipher_text, key_length, indexed_sym, threshold=.06, alphabet=A):
    
    n = len(alphabet)
    eqs = {c:[] for c in range(key_length)}
    spreaded = spread(cipher_text, key_length)
    for c,a in itertools.product(range(key_length), repeat=2):
    
        if a == c: continue
        
        column = col(spreaded, c)
        another = col(spreaded, a)
        for g in range(1,n):
            column_freqs = frequencies(column)
            another_freqs = frequencies(another)
            mci = mutual_coincidence_index(column_freqs, another_freqs, g)
            if mci > threshold: 
                eqs[c].append(tuple([a, Eq(indexed_sym[a]-indexed_sym[c],g,evaluate=True), mci]))
    
    return eqs

In [67]:
k_sym=IndexedBase('k')

eqs_dict = build_offsets_eqs(cipher_text, key_length=probing_key_length, indexed_sym=k_sym)

In [68]:
eqs_dict[0]

In [69]:
def clean_mci_info(eqs_dict):
    return {k:list(map(lambda pair: (pair[0], pair[1]), v)) for k,v in eqs_dict.items()}

In [70]:
eqs_dict_pure = clean_mci_info(eqs_dict)
eqs_dict_pure

In [71]:
def candidate_keys(eqs_dict, indexed_sym, apply_mod=False, alphabet=A):
    key_length = len(eqs_dict)
    n = len(alphabet)
    candidates = {}
    for c, eq_pairs in eqs_dict.items():
        candidates[c]=[]
        for d in range(len(alphabet)):
            key = [indexed_sym[i] for i in range(key_length)]
            key[c] = d
            for a, eq in eq_pairs:
                subs_eq = eq.subs(indexed_sym[c],d)
                key[a] = solve(subs_eq, indexed_sym[a])[0]
                if apply_mod: key[a] = key[a] % n
                    
            at_least_one_missing = next((ind for ind in key 
                                         if isinstance(ind, Indexed) or ind >= n), None)
            if not at_least_one_missing: candidates[c].append(key)
    return candidates

In [72]:
possible_keys = candidate_keys(eqs_dict_pure, k_sym)

In [73]:
def attempt_keys(candidate_keys, cipher_text, threshold=.06):
    sols = set()
    flat_keys = []
    for c, keys in candidate_keys.items(): 
        for key in keys: flat_keys.append(key)
    for key in flat_keys:
        decrypted = decrypt(cipher_text, key)
        freqs = frequencies(decrypted)
        ci = coincidence_index(freqs)
        if ci > threshold: sols.add((ci, decode(key)))
    return sols

In [74]:
sols = attempt_keys(possible_keys, cipher_text)

In [75]:
sols

{(0.07275110734757687, 'kbqacxrqycuh'),
 (0.07275110734757687, 'lcrbdysrzdvi'),
 (0.07275110734757687, 'mdscezts ewj'),
 (0.07878506600660067, ' rfqsmgxnsjx'),
 (0.07878506600660067, 'asgrtnhyotky'),
 (0.07878506600660067, 'bthsuoizpulz'),
 (0.07878506600660067, 'cuitvpj qvm '),
 (0.07878506600660067, 'jap bwqgxbtg'),
 (0.07878506600660067, 'netdf ukafxk'),
 (0.07878506600660067, 'ofuegavlbgyl'),
 (0.07878506600660067, 'pgvfhbwmchzm'),
 (0.07878506600660067, 'qhwgicxndi n'),
 (0.07878506600660067, 'rixhjdyoejao'),
 (0.07878506600660067, 'sjyikezpfkbp'),
 (0.07878506600660067, 'tkzjlf qglcq'),
 (0.07878506600660067, 'ul kmgarhmdr'),
 (0.07878506600660067, 'vmalnhbsines'),
 (0.07878506600660067, 'wnbmoictjoft'),
 (0.07878506600660067, 'xocnpjdukpgu'),
 (0.07878506600660067, 'ypdoqkevlqhv'),
 (0.07878506600660067, 'zqeprlfwmriw')}

In [76]:
for sol in sols: 
    key = sol[1]
    print("key:({})\nplaintext:\n{}\n\n".format(key, decode(decrypt(cipher_text, encode(key)))))

key:(bthsuoizpulz)
plaintext:
qebxyqqoy qflkxlcxjbafbsyixifqboyqrobx ljbpxmboeympxjlobxpqolkdivxcoljxpljbxlqebox lrkqofbpxqeykxcoljxbkdiykaxfkxcoyk bxykaxmolsbk bxfkxdbojykvxykaxf biykaxqebobxtbobxifqboyovxyasbkqrobopxjlobxayofkdxykaxy efbsbjbkqpxjlobxafpqfkdrfpebaxfqxtypxklqxfkxbkdiykaxqeyqxqebxjlpqxtlkabocrixqefkdpxtbobxmolar baxqebobxfpxklqefkdxfkxliaxbkdifpexqeyqxqyhbpxeliaxlcxqebxjfkaxtfqexqeyqxjypqbocrixykaxprzarfkdxmltboxtef expqfiixzbilkdpxqlxqebxivof yixpqykwypxlcxqebxqolrzyalropxykaxjfkkbpfkdbopxqlxtbipexoljyk bxloxqlxqebxbmf xmolpbxlcxqebxf biykaxefpqlofbpxxxqebxklojykx lknrbpqxabdoyabaxqebxbkdifpexiykdrydbxcoljxfqpxifqboyovxoykhxykaxzolrdeqxfkxyxkbtxiykdrydbxcloxqebxmlifqboxifqboyqrobxfqxafaxklqxabpqolvxfkxlkbxpbkpbxfqxafaxklqxyzplirqbivxfkqboormqxbkdifpexifqboyqrobxzrqxfqxqllhxytyvxqebxbkdifpexifqboyovxpqykayoaxykaxqeobtxqebx lrkqovxzy hxfkqlxqebx lkafqflkxlcxfqyivxzbclobxaykqbykxykyo evxlcxafyib qpxtebkxyxkbtxifqboyovxiykdrydbxtypxbpqyzifpebaxfkxqebxqfjbxlcx eyr boxqebxjf

In [77]:
decode(secret_key) == 'zqeprlfwmriw'

True

**:)**

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Vigenere cryptoanalysis</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="mailto:massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/scratchpad/blob/master/chapters/crypto/vigenere-cryptoanalysis.ipynb" rel="dct:source">https://github.com/massimo-nocentini/scratchpad/blob/master/chapters/crypto/vigenere-cryptoanalysis.ipynb</a>.